## The Observer Pattern
The Observer pattern describes a publish-subscribe relationship between a single object. The publisher, which is also known as the subject or observable, and one or more objects (the subscribers, also known as observers). In the MVC example, the publisher is the model and the subscribers are the views. The ideas behind Observer are the same as the ideas behind MVC and the separation of concerns principle, that is, to increase decoupling between the publisher and subscribers, and to make it easy to add/remove subscribers at runtime.

We generally use the Observer pattern when we want to inform/update one or more objects (observers/subscribers) about a change that happened to another object (subject/publisher/observable).

## An example

In [6]:
class Publisher(object):
    def __init__(self):
        self.observers = []
        
    def add(self, observer):
        if observer not in self.observers:
            self.observers.append(observer)
        else:
            print('Failed to add: {}'.format(observer))
    
    def remove(self, observer):
        try:
            self.observers.remove(observer)
        except ValueError:
            print('Failed to remove: {}'.format(observer))
    
    def notify(self):
        [o.notify(self) for o in self.observers]


In [7]:
class DefaultFormatter(Publisher):
    def __init__(self, name):
        super(DefaultFormatter, self).__init__()
        self.name = name
        self._data = 0

    def __str__(self):
        return "{}: '{}' has data = {}".format(type(self).__name__, self.name, self._data)

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, new_value):
        try:
            self._data = int(new_value)
        except ValueError as e:
            print('Error: {}'.format(e))
        self.notify()


In [8]:
class HexFormatter(object):
    def notify(self, publisher):
        print("{}: '{}' has now hex data = {}".format(type(self).__name__, publisher.name, hex(publisher.data)))

class BinaryFormatter(object):
    def notify(self, publisher):
        print("{}: '{}' has now bin data = {}".format(type(self).__name__, publisher.name, bin(publisher.data)))


In [9]:
def main():
    df = DefaultFormatter('test1')
    print(df)

    print()
    hf = HexFormatter()
    df.add(hf)
    df.data = 3
    print(df)

    print()
    bf = BinaryFormatter()
    df.add(bf)
    df.data = 21
    print(df)

    print()
    df.remove(hf)
    df.data = 40
    print(df)

    print()
    df.remove(hf)
    df.add(bf)

    df.data = 'hello'
    print(df)

    print()
    df.data = 15.8
    print(df)

if __name__ == '__main__':
    main()

DefaultFormatter: 'test1' has data = 0
()
HexFormatter: 'test1' has now hex data = 0x3
DefaultFormatter: 'test1' has data = 3
()
HexFormatter: 'test1' has now hex data = 0x15
BinaryFormatter: 'test1' has now bin data = 0b10101
DefaultFormatter: 'test1' has data = 21
()
BinaryFormatter: 'test1' has now bin data = 0b101000
DefaultFormatter: 'test1' has data = 40
()
Failed to remove: <__main__.HexFormatter object at 0x00000000068C7198>
Failed to add: <__main__.BinaryFormatter object at 0x00000000068C71D0>
Error: invalid literal for int() with base 10: 'hello'
BinaryFormatter: 'test1' has now bin data = 0b101000
DefaultFormatter: 'test1' has data = 40
()
BinaryFormatter: 'test1' has now bin data = 0b1111
DefaultFormatter: 'test1' has data = 15


## The observer pattern from sourcemaking

In [28]:
# https://sourcemaking.com/design_patterns/observer/python/1 (slightly modified)
"""
Define a one-to-many dependency between objects so that when one object
changes state, all its dependents are notified and updatedautomatically.
"""

import abc
import six


class Subject(object):
    """
    Know its observers. Any number of Observer objects may observe a subject.
    Send a notification to its observers when its state changes.
    """

    def __init__(self):
        self._observers = set()    # nice, avoids if observer not in self.observers ...
        self._subject_state = None

    def attach(self, observer):
        observer._subject = self
        self._observers.add(observer)
        print("subject observers are now %s" % str(self._observers))

    def detach(self, observer):
        observer._subject = None
        self._observers.discard(observer)

    def _notify(self):
        print("notifying observers...")
        for observer in self._observers:
            observer.update(self._subject_state)

    @property
    def subject_state(self):
        return self._subject_state

    @subject_state.setter
    def subject_state(self, subject_state):
        print("setting subject state to %s..." % str(subject_state))
        self._subject_state = subject_state
        self._notify()


@six.add_metaclass(abc.ABCMeta)
class Observer(object):
    """
    Define an updating interface for objects that should be notified of
    changes in a subject.
    """

    def __init__(self, name):
        self._name = name
        self._subject = None
        self._observer_state = None

    @abc.abstractmethod
    def update(self, arg):
        pass


class ConcreteObserver(Observer):
    """
    Implement the Observer updating interface to keep its state
    consistent with the subject's.
    Store state that should stay consistent with the subject's.
    """

    def update(self, arg):
        self._observer_state = arg
        print("Updating observerObj %s with state %s" % (self._name, str(self._observer_state)))


def main():
    subject = Subject()
    obs1 = ConcreteObserver("observer1")
    obs2 = ConcreteObserver("observer2")
    [subject.attach(o) for o in [obs1, obs2]]
    print("-" * 20)
    subject.subject_state = 123
    print("*" * 20)
    subject.subject_state = 456

if __name__ == "__main__":
    main()

subject observers are now set([<__main__.ConcreteObserver object at 0x0000000006A01438>])
subject observers are now set([<__main__.ConcreteObserver object at 0x0000000006A01438>, <__main__.ConcreteObserver object at 0x0000000006A01470>])
--------------------
setting subject state to 123...
notifying observers...
Updating observerObj observer1 with state 123
Updating observerObj observer2 with state 123
********************
setting subject state to 456...
notifying observers...
Updating observerObj observer1 with state 456
Updating observerObj observer2 with state 456
